In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.llms import HuggingFaceHub
from crewai import LLM
import os

# Setting up the LLMs to use for our agents.
# We can also keep different LLMs for different agents

# llm = HuggingFaceHub(
#     repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
#     huggingfacehub_api_token = os.getenv("HF_TOKEN"),
#     task = "text-generation",
# )

llm = LLM(model = "openai/gpt-4o-mini")

### Initialize the Agents

In [4]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

d:\Anaconda\envs\crewai_env\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
d:\Anaconda\envs\crewai_env\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
d:\Anaconda\envs\crewai_env\Lib\site-packages\crewai_tools\tools\selenium_scraping_tool\selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role = "Venue Coordinator",
    goal = "Identify and book an appropriate venue based on event requirements",
    tools = [search_tool, scrape_tool],
    verbose = True,
    backstory = """With a keen sense of space and understanding of event logistics, you excel at finding and securing 
    the perfect venue that fits the event's theme, size, and budget constraints."""
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role = 'Logistics Manager',
    goal = "Manage all logistics for the event including catering and equipment",
    tools = [search_tool, scrape_tool],
    verbose = True,
    backstory = """Organized and detail-oriented, you ensure that every logistical aspect of the event 
    from catering to equipment setup is flawlessly executed to create a seamless experience."""
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role = "Marketing and Communications Agent",
    goal = "Effectively market the event and communicate with participants",
    tools = [search_tool, scrape_tool],
    verbose = True,
    backstory = """Creative and communicative, you craft compelling messages and engage with potential attendees 
    to maximize event exposure and participation."""
)

### Creating Venue Pydantic Object
- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [ ]:
venue_task = Task(
    description = "Find a venue in {event_city} that meets criteria for {event_topic}.",
    expected_output = "All the details of a specifically chosen venue you found to accommodate the event.",
    human_input = True,
    output_json = VenueDetails,
    output_file = "results/venue_details.json",  # Outputs the venue details as a JSON file
    agent = venue_coordinator
)

In [17]:
logistics_task = Task(
    description = "Coordinate catering and equipment for an event with {expected_participants} participants on {tentative_date}.",
    expected_output = "Confirmation of all logistics arrangements including catering and equipment setup.",
    human_input = True,
    async_execution = True,
    agent = logistics_manager
)

In [ ]:
marketing_task = Task(
    description = "Promote the {event_topic} aiming to engage at least {expected_participants} potential attendees.",
    expected_output = "Report on marketing activities and attendee engagement formatted as markdown.",
    # async_execution = True,
    output_file = "results/marketing_report.md",  # Outputs the report as a text file
    agent = marketing_communications_agent
)

### Create the Crew
Crew assembles the agents and their tasks togethers to create a flow for execution. Order of the task is important.

In [19]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents = [venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks = [venue_task, logistics_task, marketing_task],
    verbose = True
)

2025-03-06 10:27:07,314 - 18940 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [21]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference. If any input is missing, please ask user for the input


# Agent: Venue Coordinator
## Thought: I need to gather more information about the Tech Innovation Conference, such as the expected number of attendees, preferred venue type (e.g., conference hall, auditorium), and any specific budget constraints.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference venue requirements San Francisco\"}"
## Tool Output: 
{'searchParameters': {'q': 'Tech Innovation Conference venue requirements San Francisco', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Tech. Innovation Conference 2035 | MCS2025', 'link': 'https://www.mcs2025.com/event-details/tech-innovation-conference-2035', 'snippet': "Time & Location. 01 Aug 2035, 10:00 – 18:00. Conference Center, 500 Terry Francois St. San Francisco, CA 94158

In [ ]:
from IPython.display import Markdown
Markdown("results/marketing_report.md")

```markdown
# Tech Innovation Conference 2024 Promotion Report

## Event Overview
The **Tech Innovation Conference** will take place on **September 15, 2024**. This conference aims to gather tech enthusiasts, professionals, and innovators to exchange ideas, showcase new technologies, and build networks for future collaborations.

## Target Audience
Our goal is to engage at least **500 potential attendees** from various sectors, including:
- Technology
- Startups
- Academic Institutions
- Government Agencies
- Industry Leaders

## Marketing Activities
To promote the conference and attract participants, we will implement the following marketing strategies:

1. **Social Media Campaigns**:
   - Utilize platforms like LinkedIn, Twitter, and Facebook to share engaging content, speaker announcements, and highlight attendee experiences.
   - Run targeted ad campaigns focusing on specific demographics in the tech field.

2. **Email Newsletters**:
   - Send regular updates to our mailing list, including important information about speakers, agendas, and registration details.
   - Feature testimonials and success stories from past attendees.

3. **Content Marketing**:
   - Create blog posts and articles focusing on upcoming trends, technological innovations, and the importance of networking in the tech industry.
   - Highlight key topics and speakers expected at the conference.

4. **Partnerships and Sponsorships**:
   - Collaborate with industry leaders and organizations to co-host events or promote the conference through their channels.
   - Offer sponsorship packages to businesses seeking exposure during the event.

5. **Event Listings**:
   - List the conference on various event platforms to maximize reach, including eventbrite, meetup, and specialized tech forums.

## Attendee Engagement
Engagement strategies will include:
- **Networking Opportunities**: Set aside dedicated times for networking sessions to help attendees connect with industry peers.
- **Interactive Sessions**: Host workshops, Q&A panels, and hands-on technology demonstrations to actively involve participants.
- **Feedback Mechanism**: Encourage attendees to provide feedback during and after the event to enhance the overall experience for future conferences.

## Catering and Equipment Rental
We've identified the following services to ensure a seamless experience:

### Catering Services:
- **Two Fat Men Catering**
- **Olive Garden Catering**
- **Live-Fire Catering - Rambler Events**
- **Miss D's Kitchen**
- **STF Events & Catering**

### Equipment Rental Services:
- **Supply Event Rentals**
- **Marquee Event Rentals**
- **Audio Rent KC**
- **Fun Services Midwest**
- **KCI Expo Center**

We will contact these services promptly to confirm availability and finalize arrangements to provide attendees with an enjoyable and accommodating experience.

## Conclusion
The Tech Innovation Conference promises to be a vital platform for networking and innovation in technology. With effective marketing and engaging attendee strategies, we anticipate successfully reaching our engagement goal of 500 participants.
```